<a href="https://colab.research.google.com/github/sandeep92134/PYTHON-Data-Cleaning/blob/master/Chapter%204/Exersize%206.%20Using%20k-nearest%20neighbor%20to%20find%20outliers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyod

     |████████████████████████████████| 102kB 4.5MB/s 
  Created wheel for pyod: filename=pyod-0.8.7-cp37-none-any.whl size=115979 sha256=efa28f00e3f327c1c6211a88df0a880883af94c9bf04d28e712d35614b5020ca
  Stored in directory: /root/.cache/pip/wheels/b7/8a/c9/5fe3f32692ad445fe573cf5b953d7a0d55dc1f7c8e229ebda0
Successfully built pyod


In [2]:
# import pandas, pyod, and sklearn
import pandas as pd
from pyod.models.knn import KNN
from sklearn.preprocessing import StandardScaler
pd.set_option('display.width', 80)
pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 20)
pd.options.display.float_format = '{:,.2f}'.format
covidtotals = pd.read_csv("https://raw.githubusercontent.com/sandeep92134/PYTHON-Data-Cleaning/master/Chapter%204/datasets/covidtotals.csv")
covidtotals.set_index("iso_code", inplace=True)

In [3]:
# create a standardized dataset of the analysis variables
standardizer = StandardScaler()
analysisvars = ['location','total_cases_pm','total_deaths_pm',\
  'pop_density','median_age','gdp_per_capita']
covidanalysis = covidtotals.loc[:, analysisvars].dropna()
covidanalysisstand = standardizer.fit_transform(covidanalysis.iloc[:, 1:])

In [4]:
# run the KNN model and generate anomaly scores
clf_name = 'KNN'
clf = KNN(contamination=0.1)
clf.fit(covidanalysisstand)
y_pred = clf.labels_
y_scores = clf.decision_scores_

In [5]:
# show the predictions from the model
pred = pd.DataFrame(zip(y_pred, y_scores), 
  columns=['outlier','scores'], 
  index=covidanalysis.index)
pred.sample(10, random_state=1)

,outlier,scores
iso_code,,
LBY,0,0.37
NLD,1,1.56
BTN,0,0.19
HTI,0,0.43
EST,0,0.46
LCA,0,0.43
PER,0,1.41
BRB,0,0.77
MDA,0,0.91


In [6]:
pred.outlier.value_counts()

0    157
1     18
Name: outlier, dtype: int64

In [7]:
pred.groupby(['outlier'])[['scores']].agg(['min','median','max'])

scores            
           min median  max
outlier                   
0         0.08   0.36 1.52
1         1.55   2.10 9.48

In [8]:
# show covid data for the outliers
covidanalysis.join(pred).loc[pred.outlier==1,\
  ['location','total_cases_pm','total_deaths_pm','scores']].\
  sort_values(['scores'], ascending=False)

,location,total_cases_pm,total_deaths_pm,scores
iso_code,,,,
SGP,Singapore,"5,962.73",3.93,9.48
QAT,Qatar,"19,753.15",13.19,8.00
HKG,Hong Kong,0.00,0.00,7.77
BEL,Belgium,"5,037.35",816.85,3.54
BHR,Bahrain,"6,698.47",11.17,2.84
LUX,Luxembourg,"6,418.78",175.73,2.44
ESP,Spain,"5,120.95",580.20,2.18
KWT,Kuwait,"6,332.42",49.64,2.13
GBR,United Kingdom,"4,047.40",566.97,2.10
